In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StructField, StructType
from numpy import argmax, array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow import convert_to_tensor
from tensorflow import reshape

2021-12-09 05:53:37.489965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-09 05:53:37.489980: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Input (training)

In [2]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.option("header", True).option("inferSchema", True).csv('train.csv')
L = array([list(train.asDict().values()) for train in df.collect()])
del(df)

21/12/09 05:53:39 WARN Utils: Your hostname, Caballero resolves to a loopback address: 127.0.1.1; using 10.0.0.15 instead (on interface wlp0s20f3)
21/12/09 05:53:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/09 05:53:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/09 05:53:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Preprocessing (training)

In [3]:
x_train = L[:,1:785]
y_train = [to_categorical(y, 10) for y in L[:,0]]
del(L)
x_train = [x/255. for x in x_train]

In [4]:
x_train = convert_to_tensor(x_train, dtype="float32")
x_train = reshape(x_train, (x_train.shape[0], 28, 28, 1))
y_train = convert_to_tensor(y_train, dtype="float32")

2021-12-09 05:53:54.764833: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-09 05:53:54.764872: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-09 05:53:54.764895: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Caballero): /proc/driver/nvidia/version does not exist
2021-12-09 05:53:54.765410: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training

In [5]:
modelC = Sequential()
modelC.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
modelC.add(Conv2D(64, (3, 3), activation='tanh'))
modelC.add(MaxPooling2D(pool_size=(2, 2)))
modelC.add(Dropout(0.5))
modelC.add(Flatten())
modelC.add(Dense(128, activation='relu'))
modelC.add(Dropout(0.25))
modelC.add(Dense(10, activation='softmax'))
modelC.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelC.fit(x_train, y_train, batch_size=10, epochs=20, shuffle=True)

Epoch 1/20
2185/4200 [==============>...............] - ETA: 25s - loss: 0.0485 - accuracy: 0.9236

In [ ]:
del(x_train, y_train) 

Input (predictions)

In [ ]:
df = spark.read.option("header", True).option("inferSchema", True).csv('test.csv')
x_pred = array([list(train.asDict().values()) for train in df.collect()])
del(df)

Preprocessing (predictions)

Output

In [ ]:
x_pred = [x/255. for x in x_pred]
x_pred = convert_to_tensor(x_pred, dtype="float32")
x_pred = reshape(x_pred, (x_pred.shape[0], 28, 28, 1))

Prediction

In [ ]:
y_predC = modelC.predict(x_pred, batch_size=10)
del(x_pred)
y_pred = [ int(argmax( y_predC[j] )) for j in range( len(y_predC) ) ]
del(y_predC)

Output

In [ ]:
y_pred = [(j+1, y_pred[j]) for j in range(len(y_pred))]

In [ ]:
output_df = spark.createDataFrame(spark.sparkContext.parallelize(y_pred),  StructType([
    StructField('ImageId', IntegerType(), True),
    StructField('Label', IntegerType(), True)
]))

In [ ]:

output_df.toPandas().to_csv('submission.csv', index=False)